In [4]:
import numpy as np
import csv
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 14})

from sys import stdout

from helpers import *

# Loading the data

In [5]:
ROOT_FOLDER = "../"
DATA_FOLDER = "data/processed/"

INPUT_LAB_SPEC_TRAIN = ROOT_FOLDER + DATA_FOLDER + "input_lab_spec_train.csv"
INPUT_LAB_SPEC_TEST = ROOT_FOLDER + DATA_FOLDER + "input_lab_spec_test.csv"

INPUT_LAB_CONC_TRAIN = ROOT_FOLDER + DATA_FOLDER + "input_lab_conc_train.csv"
INPUT_LAB_CONC_TEST = ROOT_FOLDER + DATA_FOLDER + "input_lab_conc_test.csv"

INPUT_AMB_SPEC = ROOT_FOLDER + DATA_FOLDER + "input_amb_spec.csv"
INPUT_AMB_CONC = ROOT_FOLDER + DATA_FOLDER + "input_amb_conc.csv"

In [9]:
# Import Laboratory data

file_lab_spec_train = pd.read_csv(INPUT_LAB_SPEC_TRAIN, header=None)
file_lab_spec_test = pd.read_csv(INPUT_LAB_SPEC_TEST, header=None)
file_lab_conc_train = pd.read_csv(INPUT_LAB_CONC_TRAIN, header=None)
file_lab_conc_test = pd.read_csv(INPUT_LAB_CONC_TEST, header=None)

X_lab_train = pd.DataFrame(file_lab_spec_train).values.T
y_lab_train = pd.DataFrame(file_lab_conc_train).values.ravel()

X_lab_test = pd.DataFrame(file_lab_spec_test).values.T
y_lab_test = pd.DataFrame(file_lab_conc_test).values.ravel()

X_lab = np.append(X_lab_train, X_lab_test, axis=0)
y_lab = np.append(y_lab_train, y_lab_test)

print(X_lab.shape)
print(y_lab.shape)

(218, 2784)
(218,)


In [7]:
# Import Ambient data

file_amb_spec = pd.read_csv(INPUT_AMB_SPEC, header=None)
file_amb_conc = pd.read_csv(INPUT_AMB_CONC, header=None)

X_amb = pd.DataFrame(file_amb_spec).values.T
y_amb = pd.DataFrame(file_amb_conc).values.ravel()

print(X_amb.shape)
print(y_amb.shape)

(2230, 2784)
(2230,)


# Upsampling laboratory train set

In [9]:
print("Number of laboratory samples : ", len(data_lab_conc))
print("Number of laboratory samples : ", len(data_amb_conc))

Number of laboratory samples :  241
Number of laboratory samples :  4304


We first start to upsample laboratory samples to obtain approximately the same number of samples between lab and amb. We need to multiply by 10 the number of laboratory samples.

Idea : Replicate the concentrations and apply a randomly disturbance within 5% of the initial concentration.

In [10]:
X_amb = pd.DataFrame(X_amb)
X_amb['concentration'] = y_amb
X_amb['category'] = 'amb'

X_lab = pd.DataFrame(X_lab)
X_lab['concentration'] = y_lab
X_lab['category'] = 'lab'

In [16]:
X = X_lab.append(X_amb)
y = X['category']
del X['category']

In [17]:
from imblearn.over_sampling import SMOTE

X_resampled, y_resampled = SMOTE().fit_resample(X, y)

# Count instances of each class
from collections import Counter

print(sorted(Counter(y_resampled).items()))

[('amb', 2230), ('lab', 2230)]


Sometimes we want to sample in an ublanaced way, so that we upsample datapoints of certain characteristic, and downsample the others. This can be achieved with weights parameter.

In [9]:
#make 10 samples without replacement
#sample1 = df.sample(n = 10, replace = False)

data_lab_conc_up = data_lab_conc.sample(frac=5, replace=True, random_state=1, \
                                        weights=data_lab_conc['(NH4)SO4'])

data_amb_conc_up = data_amb_conc.sample(frac=0.5, replace=True, random_state=1, \
                                        weights=data_amb_conc['(NH4)SO4'])